# German Credit Dataset
- 대출인지 아닌지를 예측하는 문제
- 데이터를 NB에 맞도록 간단하게 변환
- Binary 데이터로 이루어진 대출사기 데이터들로부터 대출인지 아닌지 예측

In [40]:
from pandas import Series, DataFrame
import pandas as pd
import numpy as np

In [41]:
data_url = './fraud.csv'
df = pd.read_csv(data_url, sep=',')
df.head()

,ID,History,CoApplicant,Accommodation,Fraud
0,1,current,none,own,True
1,2,paid,none,own,False
2,3,paid,none,own,False
3,4,paid,guarantor,rent,True
4,5,arrears,none,own,False


In [42]:
# ID열을 삭제해 줍니다.
del df['ID']

# label(Y_data)을 따로 저장해 줍니다.
Y_data = df.pop('Fraud')

In [43]:
# as_matrix() 함수를 통해 array형태로 변환시켜 줍니다.
# Convert the frame to its Numpy-array representation.
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.as_matrix.html

Y_data = Y_data.as_matrix()
Y_data


/Users/youngerous/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """


array([ True, False, False,  True, False,  True, False, False, False,
        True, False,  True,  True, False, False, False, False, False,
       False, False])

In [44]:
type(Y_data)

numpy.ndarray

In [45]:
df.head()

,History,CoApplicant,Accommodation
0,current,none,own
1,paid,none,own
2,paid,none,own
3,paid,guarantor,rent
4,arrears,none,own


In [46]:
# 우리가 앞으로 사용할 데이터셋입니다. 그런데 문제가 있어 보이네요...

## One-hot encoding
- 범주형 변수를 dummy 변수로 변환해주는 작업

In [51]:
# []가 두 번 들어가면 변수명에 prefix가 추가됨
dummies_history = pd.get_dummies(df[['History']])
dummies_coappli = pd.get_dummies(df[['CoApplicant']])
dummies_accommo = pd.get_dummies(df[['Accommodation']])

In [52]:
x_df = pd.concat([df.drop(['History', 'CoApplicant','Accommodation'], axis=1),dummies_history, dummies_coappli, dummies_accommo], axis=1)
x_df.head()

,History_arrears,History_current,History_none,History_paid,CoApplicant_coapplicant,CoApplicant_guarantor,CoApplicant_none,Accommodation_free,Accommodation_own,Accommodation_rent
0,0,1,0,0,0,0,1,0,1,0
1,0,0,0,1,0,0,1,0,1,0
2,0,0,0,1,0,0,1,0,1,0
3,0,0,0,1,0,1,0,0,0,1
4,1,0,0,0,0,0,1,0,1,0


10개의 feature를 얻었다.

In [53]:
x_data = x_df.as_matrix()
x_data

/Users/youngerous/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


array([[0, 1, 0, 0, 0, 0, 1, 0, 1, 0],
       [0, 0, 0, 1, 0, 0, 1, 0, 1, 0],
       [0, 0, 0, 1, 0, 0, 1, 0, 1, 0],
       [0, 0, 0, 1, 0, 1, 0, 0, 0, 1],
       [1, 0, 0, 0, 0, 0, 1, 0, 1, 0],
       [1, 0, 0, 0, 0, 0, 1, 0, 1, 0],
       [0, 1, 0, 0, 0, 0, 1, 0, 1, 0],
       [1, 0, 0, 0, 0, 0, 1, 0, 1, 0],
       [0, 1, 0, 0, 0, 0, 1, 0, 0, 1],
       [0, 0, 1, 0, 0, 0, 1, 0, 1, 0],
       [0, 1, 0, 0, 1, 0, 0, 0, 1, 0],
       [0, 1, 0, 0, 0, 0, 1, 0, 1, 0],
       [0, 1, 0, 0, 0, 0, 1, 0, 0, 1],
       [0, 0, 0, 1, 0, 0, 1, 0, 1, 0],
       [1, 0, 0, 0, 0, 0, 1, 0, 1, 0],
       [0, 1, 0, 0, 0, 0, 1, 0, 1, 0],
       [1, 0, 0, 0, 1, 0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0, 0, 1, 1, 0, 0],
       [1, 0, 0, 0, 0, 0, 1, 0, 1, 0],
       [0, 0, 0, 1, 0, 0, 1, 0, 1, 0]], dtype=uint8)

#### Q1. as_matrix() 함수를 통해 우리가 하고자 하는 것은 무엇일까요?
A. 추후 P(X|Y)를 구하는 상황에서 Y 조건 하에 True 값을 갖는 X의 합을 구하려고 한다.

In [54]:
Y_data == True # boolean index

array([ True, False, False,  True, False,  True, False, False, False,
        True, False,  True,  True, False, False, False, False, False,
       False, False])

In [55]:
len(set(Y_data))

2

## Naive bayes classifier
- P(Y)
- P(X1, X2, ... , Xn)
- P(Y|X1, X2, ... , Xn)
- P(X1|Y), P(X2|Y), ... , P(Xn|Y) 등 우리가 구해야 할 식들에 대한 아이디어가 있어야 합니다.

### P(Y1), P(Y0) 구하기

In [56]:
# P(Y1) = count(Y1) / count(Y)

P_Y_True  = sum(Y_data==True) / len(Y_data) # P(Y_true)
P_Y_False = 1 - P_Y_True                    # P(Y_false)

P_Y_True, P_Y_False

(0.3, 0.7)

- 이번 튜토리얼에서는 index를 이용합니다.
- 이해하기보다는 따라하면서 음미해보세요.

In [57]:
# y가 1일 경우, y가 0일 경우를 구해줘야 합니다.
# 이번 시간에는 np.where를 사용합니다.

In [61]:
ix_Y_True = np.where(Y_data) # Y_data가 True인 인덱스를 뽑아줍니다.
ix_Y_False = np.where(Y_data == False)

ix_Y_True, ix_Y_False

((array([ 0,  3,  5,  9, 11, 12]),),
 (array([ 1,  2,  4,  6,  7,  8, 10, 13, 14, 15, 16, 17, 18, 19]),))

In [62]:
# np.where을 사용해서 Y가 1일 때와 0일 때 각각의 인덱스 값을 얻을 수 있게 되었습니다.


### product P(X|Yc) 구하기
- product * P(X|Y1)
- product * P(X|Y2) 

In [64]:
# P(X|Y) = count(X_cap_Y) / count(Y)
p_x_y_true = x_data[ix_Y_True].sum(axis=0) / sum(Y_data == True)
p_x_y_true 

array([0.16666667, 0.5       , 0.16666667, 0.16666667, 0.        ,
       0.16666667, 0.83333333, 0.        , 0.66666667, 0.33333333])

In [71]:
p_x_y_true  = (x_data[ix_Y_True].sum(axis=0))  / sum(Y_data == True)  # sigma P(X_i|Y_true)
p_x_y_false = (x_data[ix_Y_False].sum(axis=0)) / sum(Y_data == False) # sigma P(X_i|Y_false)

p_x_y_true, p_x_y_false

(array([0.16666667, 0.5       , 0.16666667, 0.16666667, 0.        ,
        0.16666667, 0.83333333, 0.        , 0.66666667, 0.33333333]),
 array([0.42857143, 0.28571429, 0.        , 0.28571429, 0.14285714,
        0.        , 0.85714286, 0.07142857, 0.78571429, 0.14285714]))

In [72]:
# 총 10개의 값에 대해서 확률을 구해준다.

In [108]:
x_test = [0, 1, 0, 0, 0, 1, 0, 0, 1, 0]

import math

p_y_true_test  = P_Y_True  + p_x_y_true.dot(x_test)
p_y_false_test = P_Y_False + p_x_y_false.dot(x_test)

p_y_true_test, p_y_false_test

(1.6333333333333333, 1.7714285714285714)

In [109]:
p_y_true_test < p_y_false_test # 사기가 아닌 사례로 예측

True

## 2. Do Smoothing을 통해 P(Y=1|X)의 확률과 P(Y=0|X)의 확률 값을 비교하세요.

In [85]:
smoothing_p = 2
num_class = len(x_df.columns) # 10

In [117]:
# P(X|Y) = count(X_cap_Y) + k / count(Y) + k*V
p_x_y_true  = (x_data[ix_Y_True].sum(axis=0) + smoothing_p) / (sum(Y_data == True) + smoothing_p * num_class)
p_x_y_false = (x_data[ix_Y_False].sum(axis=0) + smoothing_p) / (sum(Y_data == False) + smoothing_p * num_class)

p_x_y_true, p_x_y_false

(array([0.11538462, 0.19230769, 0.11538462, 0.11538462, 0.07692308,
        0.11538462, 0.26923077, 0.07692308, 0.23076923, 0.15384615]),
 array([0.23529412, 0.17647059, 0.05882353, 0.17647059, 0.11764706,
        0.05882353, 0.41176471, 0.08823529, 0.38235294, 0.11764706]))

In [118]:
import math
x_test = [0, 1, 0, 0, 0, 1, 0, 0, 1, 0]

p_y_true_test  = P_Y_True  + p_x_y_true.dot(x_test)
p_y_false_test = P_Y_False + p_x_y_false.dot(x_test)

p_y_true_test, p_y_false_test

(0.8384615384615386, 1.3176470588235294)

In [120]:
p_y_true_test < p_y_false_test # 사기가 아닌 사례로 예측

True

변수가 많을수록 likelihood가 0이 되는 현상이 많아진다. <br/>
따라서 확률값의 분모, 분자에 smoothing을 함으로써 **약간이라도 0이 아닌 확률**을 갖게 만들었고 <br/>
그 결과, 클래스 별 확률의 차이가 더 유의미하게(판단하기 쉽게) 나타났다.


- smoothing 이전 확률값의 차: (1.6333333333333333, 1.7714285714285714) => 0.1380952380952381
- smoothing 이후 확률값의 차: (0.8384615384615386, 1.3176470588235294) => 0.4791855203619908

> 변수가 위의 예제보다 훨씬 많아질수록 smoothing의 효과가 커질 것이다